In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

# Modelo 1: Linear Regression (Múltiple)
resultados_lr = {}
lr_model = LinearRegression()
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        lr_model.fit(X_train, y_train)
        y_pred_2022 = lr_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_lr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_lr_df = pd.DataFrame(resultados_lr).T

# Modelo 2: Polynomial Regression (Regresión Polinómica)
resultados_pr = {}
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        
        pr_model = LinearRegression()
        pr_model.fit(X_train_poly, y_train)
        y_pred_2022 = pr_model.predict(X_test_poly)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_pr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_pr_df = pd.DataFrame(resultados_pr).T

# Modelo 3: Neural Network (Redes Neuronales)
resultados_nn = {}
nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        nn_model.fit(X_train, y_train)
        y_pred_2022 = nn_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_nn[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_nn_df = pd.DataFrame(resultados_nn).T

# Modelo 4: Random Forest
resultados_rf = {}
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        rf_model.fit(X_train, y_train)
        y_pred_2022 = rf_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_rf[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_rf_df = pd.DataFrame(resultados_rf).T

# Modelo 5: Gradient Boosting
resultados_gb = {}
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        gb_model.fit(X_train, y_train)
        y_pred_2022 = gb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_gb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_gb_df = pd.DataFrame(resultados_gb).T

# Modelo 6: XGBoost
resultados_xgb = {}
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        xgb_model.fit(X_train, y_train)
        y_pred_2022 = xgb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_xgb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_xgb_df = pd.DataFrame(resultados_xgb).T

# Resultados de los modelos (simula los DataFrames que imprimiste anteriormente)
resultados = {
    'Linear Regression': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}

# Crear un DataFrame para almacenar los mejores modelos para cada subproceso
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mejor Modelo', 'Diferencia %'])

# Comparar los resultados para cada subproceso
for subproceso in resultados['Linear Regression'].index:
    mejor_modelo = None
    menor_diferencia = np.inf
    
    for modelo, df in resultados.items():
        if subproceso in df.index and df.loc[subproceso, 'Diferencia %'] != 'N/A':
            diferencia = abs(df.loc[subproceso, 'Diferencia %'])
            if diferencia < menor_diferencia:
                menor_diferencia = diferencia
                mejor_modelo = modelo
    
    # Guardar el mejor modelo para el subproceso
    nueva_fila = pd.DataFrame({
        'Subproceso': [subproceso],
        'Mejor Modelo': [mejor_modelo],
        'Diferencia %': [menor_diferencia]
    })
    
    mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)
    
# Exportar como Excel los resultados
mejor_modelo_df.to_excel(r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Costos_Indicadores_Mina.xlsx', index=False)
# Mostrar los resultados
mejor_modelo_df

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

# Definir una lista de modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),  # Usaremos PolynomialFeatures más adelante
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            if nombre_modelo == 'Polynomial Regression':
                poly = PolynomialFeatures(degree=2)
                X_train = poly.fit_transform(X_train)
                X_test = poly.transform(X_test)
                modelo.fit(X_train, y_train)
            else:
                modelo.fit(X_train, y_train)
            
            y_pred_2022 = modelo.predict(X_test)
            costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] >= '2022-01-01'][subproceso].values
            
            # Calcular las diferencias y diferencias porcentuales
            diferencias = y_pred_2022 - costo_real_2022
            diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, costo_real_2022)]
            
            # Guardar resultados en el diccionario
            for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                resultados.setdefault(subproceso, {}).update({
                    f'Predicción {mes}': pred,
                    f'Costo Real {mes}': real,
                    f'Diferencia {mes}': dif,
                    f'Diferencia % {mes}': dif_pct
                })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Comparar los resultados entre modelos para cada subproceso y mes
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mes', 'Mejor Modelo', 'Diferencia %'])

for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo para el subproceso y mes
        nueva_fila = pd.DataFrame({
            'Subproceso': [subproceso],
            'Mes': [mes],
            'Mejor Modelo': [mejor_modelo],
            'Diferencia %': [menor_diferencia]
        })
        
        mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)

# Exportar como Excel los resultados
mejor_modelo_df.to_excel(r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Costos_Indicadores_Mina_Mejor_Modelo.xlsx', index=False)

# Mostrar los resultados
mejor_modelo_df


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum it

,Subproceso,Mes,Mejor Modelo,Diferencia %
0,ADM OPERACIÓN Y GESTIÓN ESP,January-2022,None,inf
1,ADM OPERACIÓN Y GESTIÓN ESP,February-2022,Linear Regression,5.738844
2,ADM OPERACIÓN Y GESTIÓN ESP,March-2022,Gradient Boosting,13.330903
3,ADM OPERACIÓN Y GESTIÓN ESP,April-2022,Random Forest,0.115401
4,ADM OPERACIÓN Y GESTIÓN ESP,May-2022,Gradient Boosting,5.585125
...,...,...,...,...
727,TRONADURA TESORO SUR,August-2022,Random Forest,4.080816
728,TRONADURA TESORO SUR,September-2022,Linear Regression,4.471137
729,TRONADURA TESORO SUR,October-2022,Gradient Boosting,14.476201
730,TRONADURA TESORO SUR,November-2022,XGBoost,1.668519
